# Cellule 1 : Monter Google Drive

# **ExoFinder : Protocole d'Initialisation et de Chargement des Données**

**Auteur(s) :** Salomon Balemba, VAN TASI MUTUNDO
**Version :** 1.0
**Date :** 4 Octobre 2025

---

### **Objectif de ce Notebook**

Ce document sert de rapport d'expérience pour la première phase de notre projet : la mise en place d'un pipeline de chargement de données robuste, sécurisé et scientifiquement validé pour le jeu de données des Objets d'Intérêt Kepler (KOI).

Chaque étape est conçue pour être reproductible et justifiée, en suivant une méthodologie prudente pour éviter les biais courants en science des données.

### **1. Connexion à l'Environnement de Données**

**Procédure :** Nous montons notre Google Drive partagé.

**Justification (`Pourquoi ?`) :**
*   **Persistance :** Les données restent disponibles entre les sessions Colab.
*   **Source Unique de Vérité :** Toute l'équipe accède au même fichier de données brutes, garantissant la cohérence de nos expériences.
*   **Performance :** Évite les téléversements répétitifs et lents à chaque session.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Après exécution, suivez le lien pour autoriser l'accès.

Mounted at /content/drive


### **2. Sélection des Caractéristiques et Protocole de Validation**

**Procédure :** Avant de charger l'intégralité du fichier, nous inspectons son en-tête pour valider les noms exacts des colonnes. Ensuite, nous définissons une liste explicite de colonnes à charger.

**Justification (`Pourquoi ?`) :**
*   **Rigueur Scientifique :** Cette approche remplace l'hypothèse (deviner les noms de colonnes) par l'observation directe (lire les vrais noms). Elle prévient les erreurs `ValueError`.
*   **Prévention de la Fuite de Données :** Nous sélectionnons *a priori* un sous-ensemble de caractéristiques basé sur une hypothèse physique, en excluant délibérément les colonnes qui pourraient contenir des "réponses" (ex: `koi_score`, les `koi_fpflag_*`).
*   **Performance et Clarté :** Charger uniquement les colonnes nécessaires rend le processus plus rapide et le DataFrame plus facile à manipuler.

---
#### **Documentation du Sous-Ensemble de Données Sélectionné**

Notre hypothèse de départ est que les caractéristiques les plus prédictives sont celles qui décrivent directement les propriétés physiques du transit et de l'étoile.

*   **Identifiants :**
    *   `kepoi_name`, `kepler_name`: Conservés pour l'identification et l'affichage des résultats. **Non utilisés pour l'entraînement.**
*   **Cible :**
    *   `koi_disposition`: La classification que nous cherchons à prédire (CONFIRMED, CANDIDATE, FALSE POSITIVE).
*   **Caractéristiques (Features) :**
    *   `koi_period`: Période orbitale. Fondamentale pour la détection d'un signal répétitif.
    *   `koi_duration`: Durée du transit. Liée à la géométrie de l'orbite.
    *   `koi_depth`: Profondeur du transit. Liée au ratio de taille planète/étoile.
    *   `koi_impact`: Paramètre d'impact. Décrit la trajectoire du transit sur le disque stellaire.
    *   `koi_srad`, `koi_slogg`, `koi_steff`: Rayon, gravité de surface et température de l'étoile. Essentiels pour contextualiser les paramètres du transit.

In [ ]:
import pandas as pd

# --- CONFIGURATION ET VÉRIFICATION ---

# Adaptez le chemin si nécessaire vers le fichier CSV complet sur votre Drive
DATA_PATH = '/content/drive/MyDrive/NASA_Space_Apps_2025/koi_data_full.csv'

# --- ÉTAPE DE VÉRIFICATION SCIENTIFIQUE ---
# Nous chargeons uniquement la première ligne pour obtenir les noms de colonnes exacts.
try:
    header_df = pd.read_csv(DATA_PATH, comment='#', nrows=1)
    print("VÉRIFICATION RÉUSSIE. Les noms de colonnes exacts trouvés dans le fichier sont :")
    actual_columns = header_df.columns.tolist()
    print(actual_columns)
except FileNotFoundError:
    print(f"ERREUR : Le fichier n'a pas été trouvé à l'emplacement : {DATA_PATH}")
    actual_columns = [] # Pour éviter une erreur plus loin

print("\n---")


# --- LA SOURCE DE VÉRITÉ : Sélection des Colonnes (VERSION CORRIGÉE) ---
# Nous utilisons maintenant les noms de colonnes validés par notre inspection.

# Colonnes d'identification (pour l'affichage)
IDENTIFIERS_COLS = [
    'kepoi_name',      # CORRIGÉ: C'était 'koi_name'
    'kepler_name'
]

# Colonne cible (ce que nous voulons prédire)
TARGET_COL = 'koi_disposition'

# Caractéristiques (Features) pour l'entraînement du modèle
FEATURES_COLS = [
    'koi_period',
    'koi_duration',
    'koi_depth',
    'koi_impact',
    'koi_srad',
    'koi_slogg',
    'koi_steff'
]

# On combine toutes les colonnes dont on a besoin pour le chargement.
COLS_TO_LOAD = IDENTIFIERS_COLS + [TARGET_COL] + FEATURES_COLS

print("Configuration des colonnes à charger MISE À JOUR et VALIDÉE :")
print(COLS_TO_LOAD)

VÉRIFICATION RÉUSSIE. Les noms de colonnes exacts trouvés dans le fichier sont :
['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag']

---
Configuration des colonnes à charger MISE À JOUR et VALIDÉE :
['kepoi_name', 'kepler_name', 'koi_disposition', 'koi_period

### **3. Chargement Sécurisé des Données**

**Procédure :** Nous utilisons `pd.read_csv` avec le paramètre `usecols` en lui passant notre liste de colonnes validée.

**Justification (`Pourquoi ?`) :**
*   **Sécurité :** Garantit que seules les colonnes que nous avons explicitement approuvées sont chargées en mémoire, éliminant tout risque d'utiliser accidentellement une colonne non désirée.
*   **Efficacité :** C'est la méthode la plus performante pour charger des sous-ensembles de gros fichiers CSV.

In [ ]:
# On charge le CSV, mais UNIQUEMENT les colonnes qui nous intéressent.
# C'est rapide, léger en mémoire, et maintenant, c'est sûr.
try:
    if actual_columns: # On ne tente de charger que si la vérification a fonctionné
        df = pd.read_csv(DATA_PATH, comment='#', usecols=COLS_TO_LOAD)
        print("\nDonnées chargées avec succès !")
        print(f"Shape du DataFrame : {df.shape}")
        df.info()
except FileNotFoundError:
    print(f"ERREUR : Le fichier n'a pas été trouvé à l'emplacement : {DATA_PATH}")
    print("Veuillez vérifier le chemin et que votre Drive est bien monté.")
except ValueError as e:
    print(f"\nERREUR DE VALEUR : {e}")
    print("Cela signifie probablement que vous n'avez pas encore mis à jour la liste des colonnes dans la cellule précédente avec les noms exacts.")



Données chargées avec succès !
Shape du DataFrame : (9564, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kepoi_name       9564 non-null   object 
 1   kepler_name      2747 non-null   object 
 2   koi_disposition  9564 non-null   object 
 3   koi_period       9564 non-null   float64
 4   koi_impact       9201 non-null   float64
 5   koi_duration     9564 non-null   float64
 6   koi_depth        9201 non-null   float64
 7   koi_steff        9201 non-null   float64
 8   koi_slogg        9201 non-null   float64
 9   koi_srad         9201 non-null   float64
dtypes: float64(7), object(3)
memory usage: 747.3+ KB
